# **R-Squared Disentangled out-of-sample**

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

from datetime import datetime, timedelta

pd.set_option('display.max_columns', 500)

### Import Data

In [2]:
dummy = pd.read_excel("C:\\Users\\Yamac\\OneDrive\\Documentos\\Datathon\\crypto_fear_greed_data.xlsx")
dummy

,date,daily_index,sentiment,weekly_mean,montly_mean
0,2023-10-23,53,Neutral,NaN,NaN
1,2023-10-22,53,Neutral,52.857143,NaN
2,2023-10-21,63,Greed,NaN,NaN
3,2023-10-20,53,Neutral,NaN,NaN
4,2023-10-19,52,Neutral,NaN,NaN
...,...,...,...,...,...
2083,2018-02-05,11,Extreme Fear,NaN,NaN
2084,2018-02-04,24,Extreme Fear,27.250000,NaN
2085,2018-02-03,40,Fear,NaN,NaN
2086,2018-02-02,15,Extreme Fear,NaN,NaN


## *R-Squared*

In [3]:
fg = dummy.drop(['sentiment', 'weekly_mean', 'montly_mean'], axis=1).set_index('date').sort_values(by='date', ascending=True)
fg['weekly_mean'] = fg['daily_index'].rolling(7).mean()
fg['sentiment'] = 0

fg.loc[(fg['weekly_mean'] <= 24), 'sentiment'] = 'Extreme Fear'
fg.loc[(fg['weekly_mean'] >24) & (fg['weekly_mean'] <= 44), 'sentiment'] = 'Fear'
fg.loc[(fg['weekly_mean'] > 44) & (fg['weekly_mean'] <= 55), 'sentiment'] = 'Neutral'
fg.loc[(fg['weekly_mean'] > 55) & (fg['weekly_mean'] <= 75), 'sentiment'] = 'Greed'
fg.loc[(fg['weekly_mean'] > 75), 'sentiment'] = 'Extreme Greed'

# fg.loc[(fg['weekly_mean'] <= 44), 'sentiment'] = 'Fear'
# fg.loc[(fg['weekly_mean'] > 44) & (fg['weekly_mean'] <= 55), 'sentiment'] = 'Neutral'
# fg.loc[fg['weekly_mean'] > 55, 'sentiment'] = 'Greed'

# fg.loc[(fg['weekly_mean'] <= 50), 'sentiment'] = 'Fear'
# fg.loc[(fg['weekly_mean'] > 50), 'sentiment'] = 'Greed'

# fg.loc[(fg['weekly_mean'] > 24) & (fg['weekly_mean'] <= 75), 'sentiment'] = 'Neutral'
# fg.loc[(fg['weekly_mean'] <= 24) | (fg['weekly_mean'] > 75), 'sentiment'] = 'Extreme'

fg

,daily_index,weekly_mean,sentiment
date,,,
2018-02-01,30,NaN,0
2018-02-02,15,NaN,0
2018-02-03,40,NaN,0
2018-02-04,24,NaN,0
2018-02-05,11,NaN,0
...,...,...,...
2023-10-19,52,48.142857,Neutral
2023-10-20,53,49.428571,Neutral
2023-10-21,63,51.714286,Neutral


In [4]:
tech = pd.read_excel("C:\\Users\\Yamac\\OneDrive\\Documentos\\Datathon\\sinais_ind_tecnicos.xlsx")
tech['Date'] = pd.to_datetime(tech['Date'])
tech

,Date,Adj Close,Volume,MA_1_9,MA_1_12,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_9,MOM_12,VOL_1_9,VOL_1_12,VOL_2_9,VOL_2_12,VOL_3_9,VOL_3_12
0,2015-03-02,274.354004,242425100,1,1,1,1,1,0,1,0,1,1,1,1,1,1
1,2015-03-09,286.393005,259502000,1,1,1,1,1,1,1,0,1,1,1,1,1,1
2,2015-03-16,267.959991,206778200,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,2015-03-23,242.712997,179497700,0,0,1,1,1,1,1,0,1,1,1,1,1,1
4,2015-03-30,260.597992,150120700,1,1,0,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,2023-10-02,27935.089844,82183775698,1,1,1,1,1,0,0,0,0,0,0,0,0,0
449,2023-10-09,27159.652344,72673654487,1,0,1,1,1,1,1,0,0,0,0,0,0,0
450,2023-10-16,29993.896484,113402383094,1,1,1,1,1,1,1,1,0,0,0,0,0,0
451,2023-10-23,33086.234375,38363572311,1,1,1,1,1,1,1,1,1,0,1,0,0,0


In [5]:
df = tech.merge(fg, how='left', left_on='Date', right_on='date')
df.dropna(inplace=True)
df.drop('daily_index', axis=1, inplace=True)
df['Date'] = df['Date'] - timedelta(3)
df

,Date,Adj Close,Volume,MA_1_9,MA_1_12,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_9,MOM_12,VOL_1_9,VOL_1_12,VOL_2_9,VOL_2_12,VOL_3_9,VOL_3_12,weekly_mean,sentiment
154,2018-02-09,10551.799805,53626569728,0,0,0,0,0,0,0,0,0,0,0,0,0,0,35.000000,Fear
155,2018-02-16,9664.730469,55388419584,0,0,0,0,0,0,0,0,0,0,0,0,0,0,61.714286,Greed
156,2018-02-23,11512.599609,48902649856,1,0,0,0,0,0,0,0,0,0,0,0,0,0,44.571429,Neutral
157,2018-03-02,9578.629883,49671590400,0,0,1,0,0,0,0,0,0,0,0,0,0,0,46.428571,Neutral
158,2018-03-09,8223.679688,42075919360,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41.714286,Fear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,2023-09-22,27983.750000,73012240734,1,1,1,0,0,0,0,0,0,0,0,0,0,0,45.857143,Neutral
448,2023-09-29,27935.089844,82183775698,1,1,1,1,1,0,0,0,0,0,0,0,0,0,47.000000,Neutral
449,2023-10-06,27159.652344,72673654487,1,0,1,1,1,1,1,0,0,0,0,0,0,0,49.428571,Neutral
450,2023-10-13,29993.896484,113402383094,1,1,1,1,1,1,1,1,0,0,0,0,0,0,46.428571,Neutral


In [6]:
rf = pd.read_excel("C:\\Users\\Yamac\\OneDrive\\Documentos\\Datathon\\WGS1MO_weekly.xls")
rf['observation_date'] = pd.to_datetime(rf['observation_date'])
rf.head(2)

,observation_date,WGS1MO,Unnamed: 2,r_free_weekly
0,2001-08-03,3.65,0.0365,0.000690
1,2001-08-10,3.61,0.0361,0.000682


In [7]:
r = df.merge(rf[['observation_date', 'r_free_weekly']], how='left', left_on='Date', right_on='observation_date')
r['return'] = r['Adj Close'].pct_change().fillna(0)
r['risk_premium'] = r['return'] - r['r_free_weekly']
rf = r.drop(index=r.index[0]).reset_index(drop=True)
rf

,Date,Adj Close,Volume,MA_1_9,MA_1_12,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_9,MOM_12,VOL_1_9,VOL_1_12,VOL_2_9,VOL_2_12,VOL_3_9,VOL_3_12,weekly_mean,sentiment,observation_date,r_free_weekly,return,risk_premium
0,2018-02-16,9664.730469,55388419584,0,0,0,0,0,0,0,0,0,0,0,0,0,0,61.714286,Greed,2018-02-16,0.000254,-0.084068,-0.084322
1,2018-02-23,11512.599609,48902649856,1,0,0,0,0,0,0,0,0,0,0,0,0,0,44.571429,Neutral,2018-02-23,0.000264,0.191197,0.190934
2,2018-03-02,9578.629883,49671590400,0,0,1,0,0,0,0,0,0,0,0,0,0,0,46.428571,Neutral,2018-03-02,0.000283,-0.167987,-0.168270
3,2018-03-09,8223.679688,42075919360,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41.714286,Fear,2018-03-09,0.000298,-0.141456,-0.141753
4,2018-03-16,8495.780273,40853020160,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33.571429,Fear,2018-03-16,0.000319,0.033087,0.032769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,2023-09-22,27983.750000,73012240734,1,1,1,0,0,0,0,0,0,0,0,0,0,0,45.857143,Neutral,2023-09-22,0.001036,0.065770,0.064735
292,2023-09-29,27935.089844,82183775698,1,1,1,1,1,0,0,0,0,0,0,0,0,0,47.000000,Neutral,2023-09-29,0.001039,-0.001739,-0.002778
293,2023-10-06,27159.652344,72673654487,1,0,1,1,1,1,1,0,0,0,0,0,0,0,49.428571,Neutral,2023-10-06,0.001043,-0.027759,-0.028801
294,2023-10-13,29993.896484,113402383094,1,1,1,1,1,1,1,1,0,0,0,0,0,0,46.428571,Neutral,2023-10-13,0.001048,0.104355,0.103307


In [8]:
dummies = pd.get_dummies(rf['sentiment'])
df = pd.concat([rf.drop('sentiment', axis=1), dummies], axis=1)
df.drop('observation_date', axis=1, inplace=True)
df

,Date,Adj Close,Volume,MA_1_9,MA_1_12,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_9,MOM_12,VOL_1_9,VOL_1_12,VOL_2_9,VOL_2_12,VOL_3_9,VOL_3_12,weekly_mean,r_free_weekly,return,risk_premium,Extreme Fear,Extreme Greed,Fear,Greed,Neutral
0,2018-02-16,9664.730469,55388419584,0,0,0,0,0,0,0,0,0,0,0,0,0,0,61.714286,0.000254,-0.084068,-0.084322,0,0,0,1,0
1,2018-02-23,11512.599609,48902649856,1,0,0,0,0,0,0,0,0,0,0,0,0,0,44.571429,0.000264,0.191197,0.190934,0,0,0,0,1
2,2018-03-02,9578.629883,49671590400,0,0,1,0,0,0,0,0,0,0,0,0,0,0,46.428571,0.000283,-0.167987,-0.168270,0,0,0,0,1
3,2018-03-09,8223.679688,42075919360,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41.714286,0.000298,-0.141456,-0.141753,0,0,1,0,0
4,2018-03-16,8495.780273,40853020160,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33.571429,0.000319,0.033087,0.032769,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,2023-09-22,27983.750000,73012240734,1,1,1,0,0,0,0,0,0,0,0,0,0,0,45.857143,0.001036,0.065770,0.064735,0,0,0,0,1
292,2023-09-29,27935.089844,82183775698,1,1,1,1,1,0,0,0,0,0,0,0,0,0,47.000000,0.001039,-0.001739,-0.002778,0,0,0,0,1
293,2023-10-06,27159.652344,72673654487,1,0,1,1,1,1,1,0,0,0,0,0,0,0,49.428571,0.001043,-0.027759,-0.028801,0,0,0,0,1
294,2023-10-13,29993.896484,113402383094,1,1,1,1,1,1,1,1,0,0,0,0,0,0,46.428571,0.001048,0.104355,0.103307,0,0,0,0,1


In [9]:
r2 = {}

n = 14  # Number of technical indicators
t = len(df)
initial_period = 150  # Initial period for the first regression
summaries = []  # To store summary objects for later use

# Arrays to store predictions and errors
predictions = np.zeros((t - initial_period-1, n))
errors = np.zeros((t - initial_period-1, n))

for i in range(3, 17):
    
    # The dependent variable (Y)
    y = df['risk_premium'][1:df.shape[0]].values
    
    # Loop to perform rolling regression and predictions
    for current_t in range(initial_period, len(y)):
        
        # The independent variable (X)
        x = df.iloc[0:current_t, i].values  # Use data from the beginning up to the current time
        x = sm.add_constant(x)  # Add the intercept
        
        # Perform regression
        model = sm.OLS(y[:current_t], x).fit()

        # Make prediction for the next value if we're not at the end of the dataset
        if current_t < len(y):
            next_x = np.array([[1, df.iloc[current_t, i]]])  # Prepare next X value as a 2D array 
            predicted_value = model.predict(next_x)[0]
            predictions[current_t - initial_period, i-3] = predicted_value
            
            # Calculate and store the error of the prediction
            actual_value = y[current_t]
            error = actual_value - predicted_value
            errors[current_t - initial_period, i-3] = error
    
    summaries.append(model.summary())   

In [10]:
# Calculate the sum of squared errors for the historical average
actual_returns = df['risk_premium'].values[initial_period+1:]
# Calculate the historical average of returns up to each prediction point
j = 0

historical_average = np.array([df['risk_premium'][:j].mean() for j in range(initial_period+1, len(df))])
# Initialize a list to store the R^2_OS for each technical indicator
r2 = {}

current_t = 150
# Loop over each of the 14 technical indicators
for i in range(3, 17):
    Ic = []
    for j in range(21, 26):
        # Calculate the sum of squared errors for the predictive model
        sse_model = np.sum(((actual_returns - predictions[:, i-3]) ** 2)*df.iloc[151:df.shape[0], j])

        # Calculate the sum of squared errors for the historical average
        sse_historical = np.sum(((actual_returns - historical_average) ** 2)*df.iloc[151:df.shape[0], j])

        # Calculate the out-of-sample R squared (R^2_OS) for this technical indicator
        r_squared_os = 1 - (sse_model / sse_historical)
        #print(f"Out-of-Sample R² for technical indicator {df.columns[i]} - {df.columns[j]}: {r_squared_os}")
        # Store the R^2_OS value in the list
        Ic.append(r_squared_os)
    r2[df.columns[i]] = Ic

    # Print the R^2_OS for the current technical indicator


In [33]:
r2_df = pd.DataFrame(r2)
r2_df = r2_df.T
r2_df.columns = ['Fear', 'Greed', 'Neutral']
r2_df

,Fear,Greed,Neutral
MA_1_9,0.029376,-0.033150,-0.030700
MA_1_12,0.002506,-0.044989,-0.113866
MA_2_9,0.005321,-0.023980,-0.023566
MA_2_12,0.002977,-0.030758,-0.055708
MA_3_9,-0.005064,-0.005164,-0.018250
MA_3_12,0.002998,-0.005468,-0.016028
MOM_9,0.025903,0.003272,-0.012121
MOM_12,-0.004294,-0.033117,-0.040460
VOL_1_9,-0.006427,-0.017217,-0.020386
VOL_1_12,-0.023052,-0.032123,-0.026863


In [11]:
r2_df = pd.DataFrame(r2)
r2_df = r2_df.T
r2_df.columns = ['Extreme Fear', 'Extreme Greed', 'Fear', 'Greed', 'Neutral']
r2_df

,Extreme Fear,Extreme Greed,Fear,Greed,Neutral
MA_1_9,0.032393,-0.012905,0.027498,-0.058856,-0.030700
MA_1_12,-0.022418,-0.020911,0.018023,-0.075560,-0.113866
MA_2_9,0.009157,-0.008560,0.002932,-0.043559,-0.023566
MA_2_12,0.006297,-0.014717,0.000909,-0.051126,-0.055708
MA_3_9,0.000683,-0.015249,-0.008641,0.007641,-0.018250
MA_3_12,0.005573,-0.011880,0.001395,0.002673,-0.016028
MOM_9,-0.041575,0.010027,0.067915,-0.005305,-0.012121
MOM_12,-0.017190,-0.013535,0.003735,-0.057979,-0.040460
VOL_1_9,0.008629,0.029061,-0.015801,-0.075976,-0.020386
VOL_1_12,-0.008338,-0.011812,-0.032213,-0.057912,-0.026863


In [12]:
for col in r2_df.columns:
    print(col, round(r2_df[col].mean(), 2))

Extreme Fear -0.0
Extreme Greed -0.01
Fear -0.01
Greed -0.03
Neutral -0.05
